In [1]:
import os
import tarfile
from collections import Counter
from urllib.request import urlretrieve
import numpy as np
from keras.layers import Input, add, concatenate, dot
from keras.layers.core import Activation, Dense, Dropout, Permute
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from keras.callbacks import TensorBoard


Using TensorFlow backend.


In [2]:
class bAbI():

    def __init__(self, use_10k=True, data_root="", padding="PAD"):
        self.url = "http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz"  # noqa
        self.vocab = []
        self.story_size = -1
        self.question_size = -1
        self.data_root = data_root
        self.use_10k = use_10k #学習用に10000件のデータを使用
        #データファイルは2,3文のストーリーの後にその解答が提示される
        # John travelled to the hallway.
        # Marry journeyed to the bathroom.
        # Where is john? hallway 1
        if not self.data_root:
            self.data_root = os.path.join(os.path.dirname("__file__"), "data")
        self.PAD = padding

    @property
    def vocab_size(self):
        return len(self.vocab)

    @property
    def data_dir(self):
        _dir = "tasks_1-20_v1-2/"
        _dir += "en-10k" if self.use_10k else "en"
        return _dir

    def _get_location(self, kind="train"):
        file_name = "qa1_single-supporting-fact_{}.txt".format(kind.lower())
        return self.data_dir + "/" + file_name

    def download(self):
        tar_file = os.path.basename(self.url)
        if os.path.exists(os.path.join(self.data_root, self.data_dir)):
            return
        if not os.path.exists(self.data_root):
            os.mkdir(self.data_root)

        file_path = os.path.join(self.data_root, tar_file)
        if not os.path.isfile(file_path):
            print("Download the bABI data...")
            urlretrieve(self.url, file_path)
        with tarfile.open(file_path, mode="r:gz") as gz:
            for kind in ["train", "test"]:
                target = self._get_location(kind)
                gz.extract(target, self.data_root)
        os.remove(file_path)
    
    #データを読み出すための処理
    def _read_qa(self, kind="train"):
        path = os.path.join(self.data_root, self._get_location(kind))
        stories, questions, answers = [], [], []
        with open(path, "r", encoding="utf-8") as f:
            story_lines = []
            for line in f:
                line = line.strip()
                index, text = line.split(" ", 1)
                if "\t" in text:
                    question, answer, _ = text.split("\t")
                    stories.append(" ".join(story_lines))
                    questions.append(question.strip())
                    answers.append(answer.strip())
                    story_lines = []
                else:
                    story_lines.append(text)

        return stories, questions, answers
    
    # 単語辞書の作成(20単語)
    def make_vocab(self):
        train_s, train_q, train_a = self._read_qa(kind="train")
        test_s, test_q, test_a = self._read_qa(kind="test")

        all_s = train_s + test_s
        all_q = train_q + test_q

        # Make vocabulary from all stories and questions
        words = []
        for s, q in zip(all_s, all_q):
            s_words = self.tokenize(s)
            if len(s_words) > self.story_size:
                self.story_size = len(s_words)

            q_words = self.tokenize(q)
            if len(q_words) > self.question_size:
                self.question_size = len(q_words)

            words += s_words
            words += q_words

        word_count = Counter(words)
        words = [w_c[0] for w_c in word_count.most_common()]
        words.insert(0, self.PAD)  # add pad
        self.vocab = words

    def tokenize(self, string):
        words = text_to_word_sequence(string, lower=True)
        return words
    
    # RNNをベースにしているため入力データは一定の長さの数値列である必要がある
    # ストーリーは最大長にそろえて、足りない分はパディング
    def get_batch(self, kind="train"):
        if self.vocab_size == 0:
            self.make_vocab()
        stories, questions, answers = self._read_qa(kind)
        s_indices = [self.to_indices(s, self.story_size) for s in stories]
        q_indices = [self.to_indices(q, self.question_size)
                     for q in questions]
        a_indices = [self.vocab.index(a) for a in answers]
        a_categorical = to_categorical(a_indices, num_classes=self.vocab_size)

        return np.array(s_indices), np.array(q_indices), a_categorical

    def to_indices(self, string, fit_length=-1):
        if self.vocab_size == 0:
            raise Exception("You have to execute make_vocab")
        words = self.tokenize(string)
        indices = [self.vocab.index(w) for w in words]
        if fit_length > 0:
            indices = indices[:fit_length]
            pad_size = fit_length - len(indices)
            if pad_size > 0:
                indices += [self.vocab.index(self.PAD)] * pad_size
        return indices
    
    # 数値列を単語列に戻すための処理
    def to_string(self, indices):
        words = [self.vocab[i] for i in indices]
        string = " ".join([w for w in words if w != self.PAD])
        return string


In [3]:
def make_model(story_size, question_size, vocab_size,
               embedding_size=64, latent_size=32, drop_rate=0.3):
    story_input = Input(shape=(story_size,))
    question_input = Input(shape=(question_size,))

    story_embed_for_a = Embedding(
                        input_dim=vocab_size,
                        output_dim=embedding_size,
                        input_length=story_size)
    question_embed = Embedding(
                        input_dim=vocab_size,
                        output_dim=embedding_size,
                        input_length=question_size)
    #ストーリと質問の2つを入力として受け取る
    #各単語は数値で表現されているためEmbedding層を使用して一定長の単語ベクトルへ変換
    story_encoder_for_a = Dropout(drop_rate)(story_embed_for_a(story_input))
    question_encoder = Dropout(drop_rate)(question_embed(question_input))

    # match story & question along seq_size to make attention on story
    # (axes=[batch, seq_size, embed_size] after encoding)
    # 変換された質問とストーリの内積をとって関連ベクトルを算出
    match = dot([story_encoder_for_a, question_encoder], axes=[2, 2])
    
    # ストーリを別のベクトル表現に変換
    story_embed_for_c = Embedding(
        input_dim=vocab_size,
        output_dim=question_size,
        input_length=story_size
    )
    story_encoder_for_c = Dropout(drop_rate)(story_embed_for_c(story_input))

    # merge match and story context
    # 関連ベクトルとマージすることで回答用の情報を作成する
    response = add([match, story_encoder_for_c])
    # (question_size x story_size) => (story_size x question_size)
    response = Permute((2, 1))(response)

    answer = concatenate([response, question_encoder], axis=-1)
    #回答用の情報を質問と結合し、LSTMに入力
    answer = LSTM(latent_size)(answer)
    answer = Dropout(drop_rate)(answer)
    answer = Dense(vocab_size)(answer)
    output = Activation("softmax")(answer)
    model = Model(inputs=[story_input, question_input], outputs=output)

    return model


In [4]:
def main(batch_size, epochs, show_result_count):
    log_dir = os.path.join(os.path.dirname("__file__"), "logs")
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    corpus = bAbI()
    corpus.download()
    corpus.make_vocab()
    train_s, train_q, train_a = corpus.get_batch(kind="train")
    test_s, test_q, test_a = corpus.get_batch(kind="test")
    print("{} train data, {} test data.".format(len(train_s), len(test_s)))
    print("vocab size is {}.".format(corpus.vocab_size))
    # 使用単語
    print("vocab:{}".format(corpus.vocab))
    # データ例
    print("Story:{}".format(train_s[0]))
    print("Story:{}".format(corpus.to_string(train_s[0])))
    print("Question:{}".format(train_q[0]))
    print("Question:{}".format(corpus.to_string(train_q[0])))
    print("Answer:{}".format(train_a[0]))
          
    model = make_model(
                corpus.story_size, corpus.question_size, corpus.vocab_size)

    # train the model
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",
                  metrics=["accuracy"])
    model.fit([train_s, train_q], [train_a],
              validation_data=([test_s, test_q], [test_a]),
              batch_size=batch_size, epochs=epochs,
              callbacks=[TensorBoard(log_dir=log_dir)]
              )

    answer = np.argmax(test_a, axis=1)
    predicted = model.predict([test_s, test_q])
    predicted = np.argmax(predicted, axis=1)

    for i in range(show_result_count):
        story = corpus.to_string(test_s[i].tolist())
        question = corpus.to_string(test_q[i].tolist())
        a = corpus.to_string([answer[i]])
        p = corpus.to_string([predicted[i]])
        ox = "o" if a == p else "x"
        print(story + "\n", question + "\n",
              "{} True: {}, Predicted: {}".format(ox, a, p))

In [5]:
if __name__ == "__main__":
    main(batch_size=64, epochs=50, show_result_count=10)

10000 train data, 1000 test data.
vocab size is 20.
vocab:['PAD', 'to', 'the', 'where', 'is', 'went', 'john', 'sandra', 'mary', 'daniel', 'journeyed', 'travelled', 'back', 'moved', 'bathroom', 'kitchen', 'office', 'hallway', 'garden', 'bedroom']
Story:[ 8 13  1  2 14  6  5  1  2 17  0  0]
Story:mary moved to the bathroom john went to the hallway
Question:[3 4 8]
Question:where is mary
Answer:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10000 samples, validate on 1000 samples
Epoch 1/50
10000/10000 [==============================] - 2s 151us/step - loss: 2.0505 - accuracy: 0.1777 - val_loss: 1.7696 - val_accuracy: 0.2370
Epoch 2/50
10000/10000 [==============================] - 1s 87us/step - loss: 1.7332 - accuracy: 0.2281 - val_loss: 1.6619 - val_accuracy: 0.2460
Epoch 3/50
10000/10000 [==============================] - 1s 87us/step - loss: 1.6755 - accuracy: 0.2404 - val_loss: 1.6610 - val_accuracy: 0.2570
Epoch 4/50
10000/10000 [==============================] - 1s 87us/step - loss: 1.6592 - accuracy: 0.2464 - val_loss: 1.6545 - val_accuracy: 0.1900
Epoch 5/50
10000/10000 [==============================] - 1s 87us/step - loss: 1.6446 - accuracy: 0.2618 - val_loss: 1.6436 - val_accuracy: 0.2600
Epoch 6/50
10000/10000 [==============================] - 1s 87us/step - loss: 1.6214 - accuracy: 0.2863 - val_loss: 1.6077 - val_accuracy: 0.3430
Epoch 7/50
10000/10000 [==============================] - 1s 88us/st